In [19]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
import os

from dotenv import load_dotenv

load_dotenv()

# Run a quick validation that we have an entry for the OPEN_API_KEY within environment variables

assert "OPENAI_API_KEY" in os.environ, "OPENAI_API_KEY environment variable must be set"

In [20]:
# Load the document
loader = TextLoader("./inputs/knowledge_base.txt")
documents = loader.load()
# Split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# Create embeddings
embeddings = OpenAIEmbeddings()
# Create a vector store
vectorstore = Chroma.from_documents(texts, embeddings)

In [21]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [23]:
from langchain.agents import Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import BaseTool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
# Define a calculator tool
class CalculatorTool(BaseTool):
    name = "Calculator"
    description = "Useful for when you need to answer questions about math"
    def _run(self, query: str) :
        try:
            return str(eval(query))
        except:
            return "I couldn't calculate that. Please make sure your input is a valid mathematical expression."

In [27]:
# Create tool instances
search = DuckDuckGoSearchRun()
calculator = CalculatorTool()
# Define the tools
tools = [Tool(name="Search",func=search.run,description="Useful for when you need to answer questions about current events"),
Tool(name="RAG-QA",func=qa.run,description="Useful for when you need to answer questions about AI and machine learning"),
Tool(name="Calculator",func=calculator._run,description="Useful for when you need to perform mathematical calculations")
]
# Initialize the agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [28]:

result = agent.run("Quelle est la différence entre l'apprentissage supervisé et non supervisé ? Et que représente 15 % de 80 ?")
print(result)



> Entering new AgentExecutor chain...
I should use RAG-QA to answer the first part of the question about the difference between supervised and unsupervised learning. For the second part, I can use the calculator to find 15% of 80.
Action: RAG-QA
Action Input: Difference between supervised and unsupervised learning
Observation: Supervised learning involves training a model on labeled data (input-output pairs) with the goal of predicting labels for new input data. Examples include image classification where each image is labeled with the correct class (dog, cat, etc.).

Unsupervised learning, on the other hand, involves training a model on unlabeled data to discover hidden patterns or groupings in the data. An example is clustering where the algorithm groups data into clusters without predefined labels.
Thought:Now, I will use the calculator to find 15% of 80.
Action: Calculator
Action Input: 15% of 80
Observation: I couldn't calculate that. Please make sure your input is a valid mathe

In [29]:
result = agent.run("Quel est l'événement actuel ?")
print(result)



> Entering new AgentExecutor chain...
I should use the Search tool to find out the current event.
Action: Search
Action Input: Current events
Observation: Live news, investigations, opinion, photos and video by the journalists of The New York Times from more than 150 countries around the world. Subscribe for coverage of U.S. and international news ... NPR news, audio, and podcasts. Coverage of breaking stories, national and world news, politics, business, science, technology, and extended coverage of major national and world events. NPR coverage of national news, U.S. politics, elections, business, arts, culture, health and science, and technology. Subscribe to the NPR Nation RSS feed. Reuters.com is your online source for the latest world news stories and current events, ensuring our readers up to date with any breaking news developments Reuters.com is your online source for the latest US news stories and current events, ensuring our readers up to date with any breaking news develop